# MODULE CS431 - CÁC KỸ THUẬT HỌC SÂU VÀ ỨNG DỤNG

**Hướng dẫn**:
- Khởi động kernel: Ở thanh menu, chọn Kernel$\rightarrow$Restart kernel.
- Hoàn tất tất cả các mục có comment `YOUR CODE HERE`
- Chạy toàn bộ các block code để kiểm tra: Ở thanh menu, chọn Cell$\rightarrow$Run All.

---

# ATTENTION IN MACHINE LEARNING TRANSLATION
## Dịch các câu từ tiếng Anh sang tiếng Việt với cơ chế Attention

Trong bài tập trước, chúng ta đã làm quen và thực hành bài tập Dịch máy từ ngôn ngữ tiếng Anh sang tiếng Việt với sự hỗ trợ của mô hình Seq2Seq. Tuy nhiên, dễ thấy rằng, kết quả đánh giá hiệu năng thu được còn hạn chế.

Vậy, làm thế nào để cải thiện khả năng dịch thuật của mô hình Dịch máy?

<table>
<tr>
  <td>
   <img width=700 src="https://www.tensorflow.org/images/tutorials/transformer/RNN%2Battention-words-spa.png"/>
  </td>
</tr>
<tr>
  <th colspan=1>Minh họa sử dụng cơ chế Attention cho Encoder/Decoder.</th>
<tr>
</table>

Cơ chế Attention là một trong những giải pháp tiềm năng để giải quyết vấn đề này thông qua quá trình tập trung vào các vùng thông tin quan trọng để giúp mô hình học tốt hơn. Với cơ chế Attention, một "từ" có thể hiệu chỉnh được trọng số của nó cho các từ khác trong câu, sao cho các từ ở gần nó thì trọng số lớn hơn, ở xa nó thì trọng số nhỏ hơn.

Mỗi "câu" sau khi đi qua bước embedding sẽ được biểu diễn dưới dạng một ma trận X(m, n) với m, n lần lượt thể hiện số từ và số chiều của mỗi vector từ được biểu diễn. Mô hình sẽ cần học được bộ trọng số (Wq, Wk, Wv) để thu được ma trận Q, K, V (Query, Key và Value). Ma trận Query và Key có tác dụng tính toán ra score cho các cặp từ. Ma trận Value sẽ dựa trên score để tính ra vector phân phối xác suất đầu ra. Hàm softmax được sử dụng để đưa giá trị score về một phân phối xác suất mà độ lớn sẽ đại diện cho mức độ attention của từ query và từ key. Như vậy mỗi một từ sẽ được biểu diễn bởi 3 vector Query, Key và Value. Bộ ba này sẽ thể hiện trọng số của mỗi từ với các từ còn lại trong câu. (Xem hình minh họa)

<table>
<tr>
  <td>
   <img width=700 src="https://phamdinhkhanh.github.io/assets/images/20190616_attention/ScoreAllWords.png"/>
  </td>
</tr>
<tr>
  <th colspan=1>Minh họa kết quả tính toán attention cho từng từ trong câu. (Blog phamdinhkhanh)</th>
<tr>
</table>

Trong bài tập này, chúng ta sẽ tạo ra một mô hình LSTM encoder-decoder sử dụng [Keras Functional API](https://www.tensorflow.org/alpha/guide/keras/functional ( với [TensorFlow](https://www.tensorflow.org/) ). Sau đó, chúng ta thực hành chuyển đổi các câu tiếng Anh sang Tiếng Việt như bài tập lần trước. Sự khác biệt ở đây chính là chúng ta sẽ cài đặt thêm một Attention Head để cải thiện hiệu năng.

Lưu ý: Sử dụng GPU để có thể huấn luyện mô hình nhanh hơn.

Chúng ta cùng thực hiện các bước khởi tạo mô hình như bài tập lần trước nhé!

## Chuẩn bị dữ liệu

### 1) Khai báo các thư viện

Chúng ta sẽ khai báo TensorFlow và Keras. Từ Keras, chúng ta sẽ sử dụng các module khác để giúp xây dựng các lớp NN, tiền xử lý dữ liệu và xây dựng các mô hình LSTM

In [ ]:
# import các thư viện cần thiết
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
import pandas as pd
from keras.models import load_model, Model
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

def fix_random_seed():
    np.random.seed(1)
    tf.random.set_seed(1)

### 2) Đọc dữ liệu


Dữ liệu có hơn 9K cặp cụm từ English-Vietnamese. Bộ dữ liệu này có thể tải tại http://www.manythings.org/anki/. Hơn thế nữa, tại đây có hơn 50 bộ câu song ngữ. Chúng ta sẽ tải bộ dữ liệu cụm từ English-Vietnamese, giải nén và đọc bằng [Pandas](https://pandas.pydata.org/).

**Các bạn có thể sử dụng các lệnh dưới đây để tải và sử dụng dữ liệu.**

In [ ]:
!wget http://www.manythings.org/anki/vie-eng.zip -O vie-eng.zip
!unzip vie-eng.zip

--2023-11-30 12:55:56--  http://www.manythings.org/anki/vie-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 380691 (372K) [application/zip]
Saving to: ‘vie-eng.zip’

vie-eng.zip         100%[===================>] 371.77K  1.76MB/s    in 0.2s    

2023-11-30 12:55:56 (1.76 MB/s) - ‘vie-eng.zip’ saved [380691/380691]

Archive:  vie-eng.zip
  inflating: _about.txt              
  inflating: vie.txt                 


Sử dụng pandas để đọc dữ liệu và thêm tên cho các cột.

In [ ]:
lines = None

# TODO: đọc dữ liệu bằng `padas.read_table`,
# gắn label cho 2 cột đầu là `eng` và `vie`,
# loại bỏ cột cuối cùng

# YOUR CODE HERE
#raise NotImplementedError()
lines = pd.read_table('/content/vie.txt',header=None)
lines = pd.DataFrame(lines)
lines.columns = ['eng','vie','att']
lines.drop(["att"], axis = 1, inplace = True)


# Chỉ lấy 1/2 bộ dữ liệu để thực nghiệm
lines = lines[:len(lines)//2]
lines.head()

,eng,vie
0,Run!,Chạy!
1,Run!,Chạy đi!
2,Help!,Giúp tôi với!
3,Help!,Cứu tôi!
4,Jump!,Nhảy đi!


In [ ]:
# Kiểm tra với bộ test
assert "DataFrame" in str(type(lines))
assert list(lines.columns) == ['eng' , 'vie']
assert len(lines.shape) == 2
assert lines.shape[1] == 2
assert lines.iloc[10]['eng']=='Hurry!'
print('Bạn đã qua bài test')

Bạn đã qua bài test


In [ ]:
# Kiểm tra với bộ test ẩn

### 3) Chuẩn bị dữ liệu đầu vào cho Encoder (`encoder_input_data`)
Mô hình Encoder sẽ được cung cấp dữ liệu đầu vào là các câu tiếng Anh đã được tiền xử lý.
Bước tiền xử lý được thực hiện như sau:

Token hóa các câu tiếng Anh từ `eng_lines`.
Xác định độ dài tối đa của câu tiếng Anh là `max_input_length`.
Thêm vào `tokenized_eng_lines` để đạt đến `max_input_length`.
Xác định kích thước từ vựng (`num_eng_tokens`) cho các từ tiếng Anh.

In [ ]:
eng_lines = list()
for line in lines.eng:
    eng_lines.append( line )

tokenizer = None
tokenized_eng_lines = None
max_input_length =  None

# TODO: Sử dụng `preprocessing.text.Tokenizer` để biến đổi các câu thành các
# tokens và lưu vào biến `tokenized_eng_lines`. Biến `max_input_length` lưu trữ
# độ dài tối đa của các tokens.
# Gợi ý: dùng 2 hàm là `fit_on_texts` và `texts_to_sequences`

# YOUR CODE HERE
#raise NotImplementedError()
eng_tokenizer = preprocessing.text.Tokenizer()
eng_tokenizer.fit_on_texts(eng_lines)
tokenized_eng_lines = eng_tokenizer.texts_to_sequences(eng_lines)
max_input_length = max(len(seq) for seq in tokenized_eng_lines)



# sử dụng pad_sequences để cố định kích thước input của encoder
padded_eng_lines = preprocessing.sequence.pad_sequences( tokenized_eng_lines, maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_eng_lines )

eng_word_dict = eng_tokenizer.word_index
num_eng_tokens = len( eng_word_dict )+1

print( 'Độ dài lớn nhất của English là {}'.format( max_input_length ))
print( 'Kích thước dữ liệu của Encoder  -> {}'.format( encoder_input_data.shape ))
print( 'Số lượng English tokens = {}'.format( num_eng_tokens))

Độ dài lớn nhất của English là 8
Kích thước dữ liệu của Encoder  -> (4702, 8)
Số lượng English tokens = 2319


### 4) Chuẩn bị dữ liệu đầu vào cho Decoder ( `decoder_input_data` )
Mô hình decoder sẽ được cung cấp dữ liệu tiếng Việt đã được tiền xử lý. Các bước tiền xử lý giống như trên. Bước này được thực hiện trước các bước khác.

Thêm nhãn `<START>` vào vị trí đầu tiên trong mỗi câu tiếng Việt.
Thêm nhãn `<END>` vào vị trí cuối cùng trong mỗi câu tiếng Việt.



In [ ]:
vie_lines = list()
# TODO: Khác với trên chúng ta phải thêm '<START> ' và ' <END>' vào
# các câu tiếng Việt để tạo thành các dữ liệu đầu vào cho decoder

# YOUR CODE HERE
#raise NotImplementedError()
for line in lines.vie:
    vie_lines.append('<START> ' + line + ' <END>')




tokenizer = None
tokenized_vie_lines = None
max_output_length = None

# TODO: Tương tự như trên, chúng ta cũng sử dụng `preprocessing.text.Tokenizer`
# để biến đổi các câu thành các tokens và lưu vào biến `tokenized_vie_lines`.
# Biến `max_output_length` lưu trữ độ dài tối đa của các tokens.
# Gợi ý: dùng 2 hàm là `fit_on_texts` và `texts_to_sequences`

# YOUR CODE HERE
#raise NotImplementedError()
vie_tokenizer = preprocessing.text.Tokenizer()
vie_tokenizer.fit_on_texts(vie_lines)
tokenized_vie_lines = vie_tokenizer.texts_to_sequences(vie_lines)
max_output_length = max(len(seq) for seq in tokenized_vie_lines)




# sử dụng pad_sequences để cố định kích thước output của decoder
padded_vie_lines = preprocessing.sequence.pad_sequences( tokenized_vie_lines , maxlen=max_output_length, padding='post' )
decoder_input_data = np.array( padded_vie_lines )
vie_word_dict = vie_tokenizer.word_index
num_vie_tokens = len( vie_word_dict )+1


print( 'Độ dài lớn nhất của tiếng việt là {}'.format( max_output_length ))
print( 'kích thước dữ liệu đầu vào của Decoder -> {}'.format( decoder_input_data.shape ))
print( 'Số lượng Vietnamese tokens = {}'.format( num_vie_tokens))

Độ dài lớn nhất của tiếng việt là 16
kích thước dữ liệu đầu vào của Decoder -> (4702, 16)
Số lượng Vietnamese tokens = 1806


In [ ]:
# Kiểm tra với bộ test
assert num_eng_tokens == 2319
assert num_vie_tokens == 1806
assert encoder_input_data.shape[1] ==  8
assert decoder_input_data.shape[1] ==  16
assert tokenized_vie_lines[50:55] == [[1, 31, 7, 1004, 4, 2],
 [1, 421, 2],
 [1, 11, 9, 14, 17, 3, 2],
 [1, 11, 9, 14, 17, 21, 2],
 [1, 717, 48, 2]]
assert tokenized_eng_lines[50:55] == [[5, 11, 190], [37, 270], [37, 985], [37, 985], [119, 107]]
print('Bạn đã qua bài test')

Bạn đã qua bài test


In [ ]:
# Kiểm tra với bộ test ẩn

### 5) Chuẩn bị nhãn cho decoder ( decoder_target_data )

Chúng ta sẽ lấy một bản sao của `tokenized_vie_lines` và sửa đổi nó như sau:

1. Chúng ta sẽ xóa nhãn `<start>` mà chúng ta đã đính kèm trước đó. Vì vậy, từ `<start>` sẽ bị xóa.

2. Chuyển đổi `padded_vie_lines` (những dòng không có nhãn `<start>`) sang các vector one-hot để huấn luyện mô hình.
Ví dụ:

```
 [ '<start>' , 'hello' , 'world' , '<end>' ]

```

sẽ trở thành

```
 [ 'hello' , 'world' , '<end>' ]

```



In [ ]:
input_decoder_target_data = list()

# TODO: chúng ta sẽ loại bỏ '<START> ' đầu tiên của các dòng trong biến `tokenized_vie_lines`
# và thêm vào `input_decoder_target_data`

# YOUR CODE HERE
#raise NotImplementedError()
for seq in tokenized_vie_lines:
    input_decoder_target_data.append(seq[1:])



padded_vie_lines = preprocessing.sequence.pad_sequences(input_decoder_target_data , maxlen=max_output_length, padding='post' )
onehot_vie_lines = utils.to_categorical( padded_vie_lines , num_vie_tokens )
decoder_target_data = np.array( onehot_vie_lines )
print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape))

Decoder target data shape -> (4702, 16, 1806)


In [ ]:
# Kiểm tra với bộ test
assert decoder_target_data.shape==(4702, 16, 1806)
# test remove start in decode_data
for i in [5, 10, 15, 20]:
    assert (sum(tokenized_vie_lines[i]) - sum(input_decoder_target_data[i])) == 1
print('Bạn đã qua bài test')

Bạn đã qua bài test


## Định nghĩa và huấn luyện các mô hình

### 1) Định nghĩa mô hình Encoder-Decoder

Giống như bài trước, mô hình sẽ có các lớp Embedding, LSTM và Dense. Tuy nhiên,decoder sẽ có thêm lớp attention, cấu hình cơ bản như sau:

* 2 lớp đầu vào: Một cho `encoder_input_data` và một cho `decoder_input_data`.
* Lớp Embedding: Để chuyển đổi các vector token sang các dense có kích thước cố định = 256. ***(Lưu ý: để tham số `mask_zero=True`, `return_state=True`, `return_sequences=True`)***
* Lớp LSTM: khởi tạo layer chứa 128 Long-Short Term cell.
Hoạt động:

1. `encoder_input_data` đầu vào lớp Embedding (`encoder_embedding`).
2. Kết quả của lớp Embedding đi đến layer LSTM để tạo ra 2 vector trạng thái (`h` và `c` là `encoder_states`)
3. Những trạng thái này được đưa vào trong layer LSTM của decoder.
4. decoder_input_data đầu vào qua lớp Embedding.
5. Embeddings đi vào lớp LSTM (mà có các trạng thái) để tạo các chuỗi.
6. Các đầu ra của lớp LSTM sẽ đi qua lớp `AttentionLayer` do chúng ta định nghĩa trước khi vào lớp cuối cùng `Dense`.




Khác với bài trước chúng ta sẽ định nghĩa các module như encoder, decoder và attention từ lớp `tf.keras.layers.Layer` và sẽ thiết kế lớp Translator để kết hợp encoder và decoder.

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_eng_tokens, units=[256, 128]):
        super(Encoder, self).__init__()

        self.embedding = tf.keras.layers.Embedding(num_eng_tokens, units[0], mask_zero=True)
        self.lstm = tf.keras.layers.LSTM(units[1], return_state=True, return_sequences=True)

    def call(self, x):

        # TODO: `x` sẽ được đi qua lớp 'self.embedding' và `self.lstm` đã tạo ra states và output.

        # YOUR CODE HERE
        #raise NotImplementedError()
        x = self.embedding(x)
        x, state_h, state_c = self.lstm(x)




        return x, [state_h , state_c]


In [ ]:
# Chúng ta sẽ kiểm tra thử input và output của encoder thông qua đoạn code dưới
x = tf.random.uniform(shape=[8, 16]) # random sample

encoder = Encoder(num_eng_tokens)
context, state = encoder(x)

print(f'input tokens, shape (batch, s): {x.shape}')
print(f'Encoder output, shape (batch, s, units): {context.shape}')

input tokens, shape (batch, s): (8, 16)
Encoder output, shape (batch, s, units): (8, 16, 128)


In [ ]:
# Kiểm tra với bộ test
assert x.shape == (8, 16)
assert context.shape == (8, 16, 128)
assert state[0].shape == ([8, 128])
print('Bạn đã qua bài test')

Bạn đã qua bài test


Tiếp theo, chúng ta sẽ thiết kế `AttentionLayer` với `num_heads=1`

In [ ]:
class AttentionLayer(tf.keras.layers.Layer):
  def __init__(self, dim):
    super().__init__()
    # TODO: Chúng ta sẽ khởi tạo các thành phần trong `AttentionLayer` bao gồm `mha`, `layernorm` và `add` từ
    # thư viện keras tf.keras.layers.MultiHeadAttention, tf.keras.layers.LayerNormalization, tf.keras.layers.Add

    self.mha = None
    self.layernorm = None
    self.add = None

    # YOUR CODE HERE
    #raise NotImplementedError()
    self.mha = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=dim, dropout=0.1)
    self.layernorm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.add = tf.keras.layers.Add()




  def call(self, x, context):

    attn_output, attn_scores = self.mha(
        query=x,
        value=context,
        return_attention_scores=True)

    attn_scores = tf.reduce_mean(attn_scores, axis=1)

    self.last_attention_weights = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [ ]:
attention_layer = AttentionLayer(256)

x = tf.random.uniform(shape=[8, 16, 512])
x_context = tf.random.uniform(shape=[8, 16, 512])

result = attention_layer(x, x_context)

print(f'Attention result, shape (batch, t, units): {result.shape}')
print(f'Attention weights, shape (batch, t, s):    {attention_layer.last_attention_weights.shape}')

Attention result, shape (batch, t, units): (8, 16, 512)
Attention weights, shape (batch, t, s):    (8, 16, 16)


In [ ]:
# Kiểm tra với bộ test
assert result.shape == (8, 16, 512)
assert attention_layer.last_attention_weights.shape == (8, 16, 16)
print('Bạn đã qua bài test')

Bạn đã qua bài test


In [ ]:
class Decoder(tf.keras.layers.Layer):

    def __init__(self, num_vie_tokens, units=[128, 256]):
        super(Decoder, self).__init__()
        # TODO: Chúng ta sẽ định nghĩa lớp `embedding`, `lstm`, `output_layer`
        # từ các class 'Embedding', 'LSTM' và `Dense` của keras.
        # `attention` được khởi tạo tù lớp `AttentionLayer` được định nghĩa trước đó
        # Lưu ý: lớp `embedding`, `lstm` được khởi tạo giống như encoder

        self.embedding = None
        self.lstm = None
        self.attention = None
        self.output_layer = None

        # YOUR CODE HERE
        #raise NotImplementedError()
        self.embedding = tf.keras.layers.Embedding(num_vie_tokens, units[0], mask_zero=True)
        self.lstm = tf.keras.layers.LSTM(units[1], return_sequences=True, return_state=True)
        self.attention = AttentionLayer(units[1])
        self.output_layer = tf.keras.layers.Dense(num_vie_tokens, activation='softmax')

    def call(self,
            context, x,
            state=None,
            return_state=False):

        x = self.embedding(x)
        x, state_h , state_c = self.lstm(x, initial_state=state)
        x = self.attention(x, context)
        output = self.output_layer(x)

        if return_state:
            return output, [state_h , state_c]
        else:
            return output

class Translator:
    @classmethod
    def add_method(cls, fun):
        setattr(cls, fun.__name__, fun)
        return fun

    def __init__(self, num_eng_tokens, num_vie_tokens, units=[256, 128]):
        self.encoder = Encoder(num_eng_tokens, units)
        self.decoder = Decoder(num_vie_tokens, units)
        self.num_eng_tokens = num_eng_tokens
        self.num_vie_tokens = num_vie_tokens

    def build(self):
        encoder_inputs = tf.keras.layers.Input(shape=( None ,))
        decoder_inputs = tf.keras.layers.Input(shape=( None ,))

        context, states = self.encoder(encoder_inputs)
        output = self.decoder(context, decoder_inputs, states)

        try:
            # Delete the keras mask, so keras doesn't scale the loss+accuracy.
            del output._keras_mask
        except AttributeError:
            pass

        self.model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )

    def train(self, encoder_input_data , decoder_input_data, decoder_target_data):
        # Các hyper-parameter ở đây được chỉnh để có thể so sánh với mô hình trước.
        # Nếu muốn, các bạn có thể tinh chỉnh để đạt hiệu suất cao hơn.
        opt=tf.keras.optimizers.RMSprop(learning_rate=0.00256, momentum=0.9)
        self.model.compile(optimizer=opt, loss='categorical_crossentropy') # CCE loss
        self.model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=64, epochs=10)

    # Load mô hình từ file
    def load(self, model_file):
        self.model = load_model(model_file)

    # Lưu mô hình hiện tại xuống file
    def save(self, model_file):
        self.model.save(model_file)

    # Tóm tắt kiến trúc mạng
    def summary(self):
        self.model.summary()

    # Thử nghiệm mô hình với dữ liệu ảnh đầu vào
    def predict(self, x_test):
        return self.model.predict(x_test)

    def tokens_to_text(self, tokens):
        words = self.id_to_word(tokens)
        result = tf.strings.reduce_join(words, axis=-1, separator=' ')
        result = tf.strings.regex_replace(result, '^ *\[START\] *', '')
        result = tf.strings.regex_replace(result, ' *\[END\] *$', '')
        return result

In [ ]:
x = tf.random.uniform(shape=[8, 16])
x_context = tf.random.uniform(shape=[8, 16, 512])
decoder = Decoder(2048, [128, 256])
out = decoder(x_context, x)

In [ ]:
# Kiểm tra với bộ test
assert out.shape == [8, 16, 2048]
print('Bạn đã qua bài test')

Bạn đã qua bài test


In [ ]:
fix_random_seed()
# TODO: Khởi tạo `alpha` từ class `Translator` được định nghĩa ở trên
# và thực hiện phương thức `build` để xây dựng mô hình.

alpha =None
# YOUR CODE HERE
#raise NotImplementedError()
alpha = Translator(num_eng_tokens,num_vie_tokens)
alpha.build()




In [ ]:
alpha.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 encoder_1 (Encoder)         ((None, None, 128),          790784    ['input_1[0][0]']             
                              [(None, 128),                                                       
                              (None, 128)])                                                       
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                              

### 2) Huấn luyện mô hình
Chúng ta sẽ huấn luyện mô hình với RMSprop optimizer và hàm cross-entropy loss

In [ ]:
alpha.train(encoder_input_data , decoder_input_data, decoder_target_data)

Epoch 1/10
74/74 [==============================] - 17s 74ms/step - loss: 4.9149
Epoch 2/10
74/74 [==============================] - 4s 54ms/step - loss: 3.7787
Epoch 3/10
74/74 [==============================] - 2s 28ms/step - loss: 3.0928
Epoch 4/10
74/74 [==============================] - 2s 24ms/step - loss: 2.5603
Epoch 5/10
74/74 [==============================] - 2s 25ms/step - loss: 2.1771
Epoch 6/10
74/74 [==============================] - 2s 23ms/step - loss: 1.9151
Epoch 7/10
74/74 [==============================] - 2s 23ms/step - loss: 1.7158
Epoch 8/10
74/74 [==============================] - 2s 28ms/step - loss: 1.5434
Epoch 9/10
74/74 [==============================] - 3s 44ms/step - loss: 1.4215
Epoch 10/10
74/74 [==============================] - 2s 23ms/step - loss: 1.3080


## Dự đoán kết quả

### 1) Sử dụng mô hình để dự đoán
Khác với bài trước, chúng ta sẽ thêm 2 hàm (`translate` và `plot_attention`) vào Translator để thực hiện các tác vụ dự đoán và trực quan hóa.

In [ ]:
map_i2w = {val: key for key, val in vie_word_dict.items()}
@Translator.add_method
def translate(self, input, return_attention=False):
    enc_model, dec_model = self.encoder, self.decoder
    context, states_values = enc_model(input)
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = vie_word_dict['start']
    stop_condition = False
    decoded_translation = []
    attentive_scores = []
    while not stop_condition:
        dec_outputs , [h , c] = dec_model(context, empty_target_seq, states_values, return_state=True)
        attentive_scores.append(dec_model.attention.last_attention_weights[0])
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        word =  map_i2w.get(sampled_word_index, '')
        decoded_translation.append( word )
        sampled_word = word
        if sampled_word == 'end' or len(decoded_translation) > max_output_length:
            stop_condition = True


        empty_target_seq = np.zeros( ( 1 , 1 ) )
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ]
    if return_attention:
        return decoded_translation[:-1], attentive_scores[:-1]

    return decoded_translation[:-1]

@Translator.add_method
def plot_attention(self, text, **kwargs):
    assert isinstance(text, str)

    input = eng_tokenizer.texts_to_sequences([text])
    input = preprocessing.sequence.pad_sequences(input, maxlen=max_input_length , padding='post' )
    output, attention = self.translate(input, return_attention=True)
    attention= tf.concat(attention, 0)

    context = text.split()
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1)

    ax.matshow(attention, cmap='viridis', vmin=0.0)

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + context, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + output, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    ax.set_xlabel('Input text')
    ax.set_ylabel('Output text')

### 2) Thực hiện một số bản dịch

1. Đầu tiên, sử dụng `enc_model` để dự đoán các giá trị trạng thái cho một câu tiếng Anh.
2. Thiết lập các giá trị trạng thái trong LSTM của decoder.
3. Sau đó, tạo ra một dãy có chứa phần tử `<start>`.
4. Nhập dãy này vào `dec_model`.
5. Thay thế phần tử `<start>` với phần tử được dự đoán bởi `dec_model` và  cập nhật các giá trị trạng thái.
6. Thực hiện các bước trên liên tục cho đến khi gặp phần tử `<end>` hoặc đạt đến độ dài tối đa của chuỗi.




In [ ]:
from tqdm import tqdm
enc_model , dec_model = alpha.encoder, alpha.decoder
num_sample = encoder_input_data.shape[0]
predict_translation = []
eng_sentence = []
num_sample = 800
for i in tqdm(range( num_sample ) ):
    decoded_translation = alpha.translate(encoder_input_data[ i ][None,...])
    eng_sentence.append(encoder_input_data[ i ])
    predict_translation.append( decoded_translation[:-1] )

100%|██████████| 800/800 [01:47<00:00,  7.45it/s]


In [ ]:
value2key = {val:key for key, val in vie_tokenizer.word_index.items()}
gt = [[[value2key[i] for i in line[1:-1]]] for line in tokenized_vie_lines[:num_sample]]
predict = [f for f in predict_translation]

In [ ]:
for i in [20, 50, 70]:
    print(f'input: {eng_lines[i]}, predict: {predict[i]}, label: {gt[i][0]}')

input: Really?, predict: ['thực', 'sự', 'thực', 'sự', 'thực', 'thực', 'sự', 'thực', 'thực', 'thực', 'thực', 'thực', 'thực', 'thực', 'thực'], label: ['thật', 'á']
input: Is it bad?, predict: ['nó', 'đang'], label: ['nó', 'có', 'tồi', 'không']
input: I did that., predict: ['tôi', 'đã', 'làm', 'điều'], label: ['tôi', 'đã', 'làm', 'đấy']


In [ ]:
assert len(predict) == len(gt), f'{len(predict)}, {len(gt)}'

In [ ]:
predict = [f for f in predict_translation]

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
# references = [[['this', 'is' 'cat']]]
# candidates = [['this', 'is', 'a', 'cat'],]
references = gt
candidates = predict
score = corpus_bleu(references, candidates, weights=(1, 0, 0, 0))
print('giá trị bleu score là', score)

giá trị bleu score là 0.5512268762064348


In [ ]:
options = ['10%-19%', '20%-29%', '30%-39%', '40%-49%', '50%-59%', '60%-69%', '70%-79%', '80%-89%', '90%-99%', '100%']
your_choice = None

# YOUR CODE HERE
#raise NotImplementedError()
your_choice = 4

print('giá trị bleu score đạt khoảng', options[your_choice])

giá trị bleu score đạt khoảng 50%-59%


In [ ]:
your_choice = None
answers = {
    1: 'cao hơn',
    2: 'ngang bằng',
    3: 'thấp hơn',
}
# YOUR CODE HERE
#raise NotImplementedError()
your_choice = 1

print('So với seq2seq, mô hình seq2seq với attention đạt hiệu suất ', answers[your_choice])

So với seq2seq, mô hình seq2seq với attention đạt hiệu suất  cao hơn


Chúng ta có thể kiểm tra và trực quan hóa attention map bằng cách nhập các câu tiếng anh vào biến `human_input`.

Lưu ý: ô này không tính điểm.


In [ ]:
# YOUR CODE HERE
#raise NotImplementedError()